In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [2]:
trainingData = "ex_data/trainingData.csv"
unlabeledData = "ex_data/unlabeledData.csv"
outputPath = "predictionsLR/"

In [3]:
# *************************
# Training step
# *************************

In [4]:
# Create a DataFrame from trainingData.csv
# Training data in raw format
trainingData = spark.read.load(trainingData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [5]:
#trainingData.show()

In [6]:
# Define an assembler to create a column (features) of type Vector containing the double values associated with 
# columns attr1, attr2, attr3
assembler = VectorAssembler(inputCols=["attr1", "attr2", "attr3"],\
                            outputCol="features")

In [7]:
# Apply the assembler to create column features for the training data
trainingDataDF = assembler.transform(trainingData)

In [8]:
#trainingDataDF.printSchema()
#trainingDataDF.show()

In [9]:
# Create a LogisticRegression object.  
# LogisticRegression is an Estimator that is used to 
# create a classification model based on logistic regression.
lr = LogisticRegression()

In [10]:
# We can set the values of the parameters of the 
# Logistic Regression algorithm using the setter methods.
# There is one set method for each parameter
# For example, we are setting the number of maximum iterations to 10
# and the regularization parameter. to 0.0.1
lr.setMaxIter(10)
lr.setRegParam(0.01)

LogisticRegression_65047021549c

In [11]:
# Train a logistic regression model on the training data
classificationModel = lr.fit(trainingDataDF)

In [12]:
# Now, the classification model can be used to predict the class label
# of new unlabeled data

In [13]:
# Create a DataFrame from unlabeledData.csv
# Unlabeled data in raw format
unlabeledData = spark.read.load(unlabeledData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [14]:
unlabeledData.printSchema()
unlabeledData.show()

root
 |-- label: string (nullable = true)
 |-- attr1: double (nullable = true)
 |-- attr2: double (nullable = true)
 |-- attr3: double (nullable = true)

+-----+-----+-----+-----+
|label|attr1|attr2|attr3|
+-----+-----+-----+-----+
| null| -1.0|  1.5|  1.3|
| null|  3.0|  2.0| -0.1|
| null|  0.0|  2.2| -1.5|
+-----+-----+-----+-----+



In [15]:
# Apply the same assembler we created before also on the unlabeled data to create the features column
unlabeledDataDF = assembler.transform(unlabeledData)

In [16]:
unlabeledDataDF.printSchema()
unlabeledDataDF.show()

root
 |-- label: string (nullable = true)
 |-- attr1: double (nullable = true)
 |-- attr2: double (nullable = true)
 |-- attr3: double (nullable = true)
 |-- features: vector (nullable = true)

+-----+-----+-----+-----+--------------+
|label|attr1|attr2|attr3|      features|
+-----+-----+-----+-----+--------------+
| null| -1.0|  1.5|  1.3|[-1.0,1.5,1.3]|
| null|  3.0|  2.0| -0.1|[3.0,2.0,-0.1]|
| null|  0.0|  2.2| -1.5|[0.0,2.2,-1.5]|
+-----+-----+-----+-----+--------------+



In [18]:
# Make predictions on the unlabled data using the transform() method of the trained classification model
# transform uses only the content of 'features' to perform the predictions
predictionsDF = classificationModel.transform(unlabeledDataDF)

In [19]:
predictionsDF.printSchema()
predictionsDF.show()

root
 |-- label: string (nullable = true)
 |-- attr1: double (nullable = true)
 |-- attr2: double (nullable = true)
 |-- attr3: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+-----+-----+-----+-----+--------------+--------------------+--------------------+----------+
|label|attr1|attr2|attr3|      features|       rawPrediction|         probability|prediction|
+-----+-----+-----+-----+--------------+--------------------+--------------------+----------+
| null| -1.0|  1.5|  1.3|[-1.0,1.5,1.3]|[-6.5872014439355...|[0.00137599470692...|       1.0|
| null|  3.0|  2.0| -0.1|[3.0,2.0,-0.1]|[3.98018281942565...|[0.98166040093741...|       0.0|
| null|  0.0|  2.2| -1.5|[0.0,2.2,-1.5]|[-6.3765177028604...|[0.00169814755783...|       1.0|
+-----+-----+-----+-----+--------------+--------------------+--------------------+----------+



In [20]:
# The returned DataFrame has the following schema (attributes)
# - features: vector (values of the attributes)
# - label: double (value of the class label)
# - rawPrediction: vector (nullable = true)
# - probability: vector (The i-th cell contains the probability that the current record belongs to the i-th class
# - prediction: double (the predicted class label)

# Select only the original features (i.e., the value of the orginal attributes attr1, attr2, attr3) and 
# the predicted class for each record
predictions = predictionsDF.select("attr1", "attr2", "attr3", "prediction")

In [21]:
# Save the result in an HDFS output folder
predictions.show()

+-----+-----+-----+----------+
|attr1|attr2|attr3|prediction|
+-----+-----+-----+----------+
| -1.0|  1.5|  1.3|       1.0|
|  3.0|  2.0| -0.1|       0.0|
|  0.0|  2.2| -1.5|       1.0|
+-----+-----+-----+----------+



In [22]:
# Save the result in an HDFS output folder
predictions.write.csv(outputPath, header="true")